# M44

Small open economy model with a (auction) quota

|Markets   |     X1  |    X2  |    E1  |    M2  |    W   |    CONS  |
|----------|---------|--------|--------|--------|--------|----------|
|P1        |     150 |        |   -50  |        |   -100 |          |     
|P2        |         |    40  |        |    60  |   -100 |          |      
|PL        |    -100 |   -20  |        |        |        |     120  |
|PK        |     -50 |   -20  |        |        |        |      70  |
|PW        |         |        |        |        |    200 |    -200  |
|PFX       |         |        |    50  |   -50  |        |          |      
| T (tar rev)|       |        |        |   -10  |        |     10   |

In [3]:
using MPSGE

In [14]:
M44 = MPSGEModel()


initial_tax = .2

@parameters(M44, begin
    PE1, 1,     (description = "Export price of good 1",)
    PE2, 1/(1+initial_tax)*.99,   (description = "Export price of good 2",)
    PM1, 1.01,  (description = "Import price of good 1",)
    PM2, 1/(1+initial_tax),     (description = "Import price of good 1",)    
    TM2, initial_tax, (description = "Tariff for good 2",)
end)

@sectors(M44, begin
    X1,	(description = "Production index for good 1",)
    X2,	(description = "Production index good 2",)
    E1,	(description = "Export level of good 1",)
    E2,	(description = "Export level of good 2",)
    M1,	(description = "Import level of good 1",)
    M2,	(description = "Import level of good 2",)
    W,	(description = "Welfare index ",)
end)
@commodities(M44, begin
    P1,	(description = "Price index for good 1",)
    P2,	(description = "Price index for good 1",)
    PFX,	(description = "Read exchange rate index",)
    PW,	(description = "Welfare price index",)
    PL,	(description = "Wage index",)
    PK,	(description = "Capital rental index",)
end)
@consumers(M44, begin
    CONS,	(description = "Income level for representative agent",)
end)

@auxiliary(M44, Q)


@production(M44, X1, [s=1,t=0],begin
    @output(P1, 150, t)
    @input(PL, 100,s)
    @input(PK, 50, s)
end)

@production(M44, X2, [s=1,t=0], begin
    @output(P2, 40, t)
    @input(PL, 20, s)
    @input(PK, 20, s)
end)

@production(M44, E1, [s=0,t=0], begin
    @output(PFX, 50*PE1, t)
    @input(P1, 50, s)
end)

@production(M44, M2, [s=0,t=0], begin
    @output(P2, 60, t)
    @input(PFX, 60*PM2, s, taxes = [Tax(CONS, Q)]) # what is N?
end)

@production(M44, E2, [s=0,t=0], begin
    @output(PFX, 60*PE2, t)
    @input(P2, 60, s)
end)

@production(M44, M1, [s=0,t=0], begin
    @output(P1, 50, t)
    @input(PFX, 50*PM1, s)
end)

@production(M44, W, [s=1,t=0], begin
    @output(PW, 200, t)
    @input(P1, 100, s)
    @input(P2, 100, s)
end)

@demand(M44, CONS, begin
    @final_demand(PW, 200)
end,begin
    @endowment(PL, 120)
    @endowment(PK, 70)
end);

@aux_constraint(M44, Q,
    1 - M2
)

ScalarAuxConstraint(Q, -M2 + 1)

# Benchmark

In [15]:
fix(PW, 1)

set_start_value(E2, 0)
set_start_value(M1, 0)
set_start_value(Q, 0.2)

solve!(M44, cumulative_iteration_limit = 0)
generate_report(M44)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_F6C2.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 14

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (zero_profit[X1)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Solver Status: LOCALLY_SOLVED
Model Status: FEASIBLE_POINT

Row,var,value,margin
,GenericV…,Float64,Float64
1,X1,1.0,0.0
2,X2,1.0,0.0
3,W,1.0,0.0
4,M2,1.0,0.0
5,PL,1.0,0.0
6,PW,1.0,0.0
7,P1,1.0,0.0
8,PK,1.0,0.0
9,P2,1.0,0.0


# Counterfactual

In [16]:
fix(Q,0)


solve!(M44)
generate_report(M44)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_F9B1.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 13

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.1400e+01             0.0e+00 (zero_profit[M2)
    1    10     0    11 2.1359e+01  1.9e-03    0.0e+00 (zero_profit[M2)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0    11     2 2.1359e+01           I 0.0e+00 1.9e+01 (zero_profit[M2)
    1     2    12     3 1.4242e+00  1.0e+00 SM 0.0e+00 1.1e+00 (zero_profit[W)
    2     1    13     4 4.3077e-03  1.0e+00 SO 0.0e+00 3.5e-03 (market_clearanc)
    3     1    14     5 7.5198e-08  1.0e+00 SO 0.0e+00 5.6e-08 (zero_profit[X1)

Major Iterations. . . . 3
Minor Iterations. . . . 4
Res

Row,var,value,margin
,GenericV…,Float64,Float64
1,X1,1.26327,-5.62526e-8
2,X2,0.0,6.19155
3,W,1.03788,0.0
4,M2,1.89491,-7.10543e-15
5,PL,1.1532,4.68214e-8
6,PW,1.0,-1.42126e-7
7,P1,1.09545,0.0
8,PK,0.988461,1.72682e-8
9,P2,0.912871,2.84217e-14
